In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import sys
sys.path.append('/home/rustam/hyperspecter_segmentation/makitorch')

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import IncrementalPCA
from hsi_dataset_api import HsiDataset

from makitorch.dataloaders.HsiDataloader import HsiDataloader

from tqdm import tqdm

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_indices = np.load('data/kfold1_indx_train.npy')

In [4]:
BATCH_SIZE = 32
dataset_hsi_loader = HsiDataloader(
    '/raid/rustam/hyperspectral_dataset/new_cropped_hsi_data', 
    indices=train_indices,
    shuffle_data=True,
)

data_loader = torch.utils.data.DataLoader(dataset_hsi_loader, batch_size=BATCH_SIZE)

In [5]:
ipca = IncrementalPCA(n_components=17, whiten=True, batch_size=BATCH_SIZE)

iterator = tqdm(data_loader)

for img_batch, _ in iterator:
    # (N, C, H, W) --> (N, H, W, C)
    img_batch = img_batch.permute(0, 2, 3, 1)
    # (N, H, W, C) --> (N * H * W, C)
    img_batch = img_batch.reshape(-1, img_batch.shape[-1]).numpy()
    ipca.partial_fit(img_batch)
iterator.close()

8it [22:09, 166.19s/it]


In [6]:
ipca.components_

array([[ 0.00664487,  0.00698243,  0.00731702, ...,  0.01326868,
         0.01358837,  0.0139171 ],
       [ 0.01829497,  0.01998002,  0.02136745, ...,  0.00230522,
         0.00222103,  0.00209617],
       [-0.02799355, -0.02930655, -0.03040635, ..., -0.03724863,
        -0.03653305, -0.03615084],
       ...,
       [ 0.01437053,  0.01375595,  0.01875746, ..., -0.03346951,
        -0.03268389, -0.02393122],
       [-0.07155322, -0.06864634, -0.06060989, ..., -0.08223515,
        -0.07973968, -0.07791437],
       [ 0.07024197,  0.07552695,  0.08006636, ..., -0.04590988,
        -0.04802529, -0.05220174]])

In [7]:
ipca.components_.shape

(17, 237)

In [8]:
ipca.explained_variance_

array([1.77183243e+05, 3.82642525e+04, 2.89676906e+03, 1.18570837e+03,
       1.11131965e+03, 5.76405503e+02, 3.32988200e+02, 2.19422378e+02,
       1.49255630e+02, 1.41434758e+02, 7.73961385e+01, 5.84206402e+01,
       4.31785834e+01, 3.16236142e+01, 2.13325104e+01, 2.10315602e+01,
       1.80047197e+01])

In [9]:
ipca.mean_ 

array([  4.80992745,   5.09708915,   5.35325398,   5.68790162,
         6.02362356,   6.32627559,   6.68366381,   7.03335737,
         7.25175165,   7.4802683 ,   7.7820673 ,   8.09130239,
         8.36870548,   8.59310481,   8.774696  ,   8.91829384,
         9.10780792,   9.42017749,   9.80533552,  10.21830929,
        10.61795905,  10.82332271,  10.77663403,  10.68117908,
        10.99071614,  11.55627802,  12.13291299,  12.65484664,
        13.05610256,  13.27352378,  13.42721788,  13.71519524,
        14.17433812,  14.72565956,  15.31428418,  15.8381623 ,
        16.20453666,  16.69989195,  17.56469612,  18.63050296,
        19.66052322,  20.73500225,  21.99046769,  23.27012655,
        24.35160774,  25.26446582,  25.89704789,  26.23772341,
        26.60433157,  26.95160314,  27.30930305,  27.88480701,
        28.66981087,  29.4014629 ,  29.87071768,  30.07454945,
        30.11349967,  29.88142977,  29.40048946,  28.87040057,
        28.47889136,  28.44256091,  28.63273582,  28.80

In [12]:
np.save('data/kfold1_PcaExplainedVariance_.npy', ipca.explained_variance_)
np.save('data/kfold1_PcaMean.npy', ipca.mean_)
np.save('data/kfold1_PcaComponents.npy', ipca.components_)

In [13]:
ipca.explained_variance_

array([1.77183243e+05, 3.82642525e+04, 2.89676906e+03, 1.18570837e+03,
       1.11131965e+03, 5.76405503e+02, 3.32988200e+02, 2.19422378e+02,
       1.49255630e+02, 1.41434758e+02, 7.73961385e+01, 5.84206402e+01,
       4.31785834e+01, 3.16236142e+01, 2.13325104e+01, 2.10315602e+01,
       1.80047197e+01])